<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

In [4]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [6]:
tasks = 0

__Все задания нужно выполнить только с помощью sql__

__Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)__

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
          SUM(IIF(CRIM is NULL, 1, 0)) AS cnt_CRIM,
          SUM(IIF(ZN is NULL, 1, 0)) AS cnt_ZN,
          SUM(IIF(INDUS is NULL, 1, 0)) AS cnt_INDUS,
          SUM(IIF(CHAS is NULL, 1, 0)) AS cnt_CHAS,
          SUM(IIF(NOX is NULL, 1, 0)) AS cnt_NOX
    FROM boston
    

    ''',
    con,
)

,cnt_CRIM,cnt_ZN,cnt_INDUS,cnt_CHAS,cnt_NOX
0,0,0,0,0,0


Пустых значений нет ↑

__Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)__

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
          COUNT(DISTINCT CRIM) AS dist_CRIM,
          COUNT(DISTINCT ZN) AS dist_ZN,
          COUNT(DISTINCT INDUS) AS dist_INDUS,
          COUNT(DISTINCT CHAS) AS dist_CHAS,
          COUNT(DISTINCT NOX) AS dis_NOX
    FROM boston
    limit 5
    ''',
    con,
)

,dist_CRIM,dist_ZN,dist_INDUS,dist_CHAS,dis_NOX
0,504,26,76,2,81


__Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.__

Напишите какой вывод можно сделать по данным в этих колонках

Сначала найдём медианы в каждой колонке и далее сравним их с минимальными значениями каждой из этих колонок

In [9]:
tasks+=1
pd.read_sql(
    '''
    WITH median_CRIM AS
    (SELECT 
           CRIM AS CRIM
    FROM boston
    ORDER BY CRIM
    LIMIT 2
    OFFSET (SELECT (COUNT(CRIM) - 1) / 2
              FROM boston)
    ),
    
    median_ZN AS
    (SELECT 
           ZN
     FROM boston
     ORDER BY ZN
     LIMIT 2
     OFFSET (SELECT (COUNT(ZN) - 1) / 2
              FROM boston)
    ),
    
    median_INDUS AS
    (SELECT 
           INDUS
     FROM boston
     ORDER BY INDUS
     LIMIT 2
     OFFSET (SELECT (COUNT(INDUS) - 1) / 2
              FROM boston)
    ),
    
    median_CHAS AS
    (SELECT 
           CHAS
     FROM boston
     ORDER BY CHAS
     LIMIT 2
     OFFSET (SELECT (COUNT(CHAS) - 1) / 2
              FROM boston)
    ),
    
    median_NOX AS
    (SELECT 
           NOX
     FROM boston
     ORDER BY NOX
     LIMIT 2
     OFFSET (SELECT (COUNT(NOX) - 1) / 2
              FROM boston)
    )
    
    SELECT DISTINCT 
           median_CRIM.CRIM AS mediana_CRIM,
           MIN(boston.CRIM) AS min_CRIM,
           median_ZN.ZN AS mediana_ZN,
           MIN(boston.ZN) AS min_ZN,
           median_INDUS.INDUS AS mediana_INDUS,
           MIN(boston.INDUS) AS min_INDUS,
           median_CHAS.CHAS AS mediana_CHAS,
           MIN(boston.CHAS) AS min_CHAS,
           median_NOX.NOX AS mediana_NOX,
           MIN(boston.NOX) AS min_NOX
           
    FROM boston JOIN median_CRIM 
                JOIN median_ZN 
                JOIN median_INDUS
                JOIN median_CHAS
                JOIN median_NOX
    ''',
    con,
)


,mediana_CRIM,min_CRIM,mediana_ZN,min_ZN,mediana_INDUS,min_INDUS,mediana_CHAS,min_CHAS,mediana_NOX,min_NOX
0,0.25387,0.00632,0.0,0.0,9.69,0.46,0.0,0.0,0.538,0.385


↑ В колонках ZN, CHAS медиана равна минимальному значению. 

Это значит, что минимальных значений в этих колонках больше половины. То есть в Бостоне доля жилой земли и наличие водоёмов - минимально

__Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.__

Сначала решала задачу с помощью CTE, потом уже с помощью оконных функций, оставила для себя оба варианта

Сначала найдём среднее количество комнат в самых дорогих домах, потом среднее количество комнат в 25,50,100,200,300 самых дешевых домах и вычтем одно из другого: 

1-й способ:

In [10]:
tasks+=1
pd.read_sql(
    '''
    WITH max_RM AS 
    (SELECT 
          AVG(RM) AS avg_MAX_RM
    FROM boston
    WHERE MEDV in (SELECT 
                        MAX(MEDV) AS max_MEDV
                   FROM boston
                   )
    ),
    
    avg_min_RM_25 AS 
    (SELECT AVG(RM) AS avg_RM_25
     FROM boston
     WHERE MEDV in (SELECT MEDV
                    FROM boston
                    ORDER BY MEDV
                    LIMIT 25
                    )
         
     ),
     
    avg_min_RM_50 AS 
    (SELECT AVG(RM) AS avg_RM_50
     FROM boston
     WHERE MEDV in (SELECT MEDV
                    FROM boston
                    ORDER BY MEDV
                    LIMIT 50
                    )
         
     ),
     
    avg_min_RM_100 AS 
    (SELECT AVG(RM) AS avg_RM_100
     FROM boston
     WHERE MEDV in (SELECT MEDV
                    FROM boston
                    ORDER BY MEDV
                    LIMIT 100
                    )
         
     ),
     
    avg_min_RM_200 AS 
    (SELECT AVG(RM) AS avg_RM_200
     FROM boston
     WHERE MEDV in (SELECT MEDV
                    FROM boston
                    ORDER BY MEDV
                    LIMIT 200
                    )
         
     ),
     
    avg_min_RM_300 AS 
    (SELECT AVG(RM) AS avg_RM_300
     FROM boston
     WHERE MEDV in (SELECT MEDV
                    FROM boston
                    ORDER BY MEDV
                    LIMIT 300
                    )
         
     )
    
    SELECT DISTINCT 
                avg_RM_25,
                avg_RM_50,
                avg_RM_100,
                avg_RM_200,
                avg_RM_300,
                avg_MAX_RM,
                avg_MAX_RM-avg_RM_25 AS dif_avg_25,   
                avg_MAX_RM-avg_RM_50 AS dif_avg_50,
                avg_MAX_RM-avg_RM_100 AS dif_avg_100,
                avg_MAX_RM-avg_RM_200 AS dif_avg_200,
                avg_MAX_RM-avg_RM_300 AS dif_avg_300                 
    FROM boston JOIN max_RM
                JOIN avg_min_RM_25
                JOIN avg_min_RM_50
                JOIN avg_min_RM_100
                JOIN avg_min_RM_200
                JOIN avg_min_RM_300
       
    ''',
    con,
)

,avg_RM_25,avg_RM_50,avg_RM_100,avg_RM_200,avg_RM_300,avg_MAX_RM,dif_avg_25,dif_avg_50,dif_avg_100,dif_avg_200,dif_avg_300
0,5.719,5.757608,5.882832,5.913284,5.973884,7.484,1.765,1.726392,1.601168,1.570716,1.510116


2-й способ:

In [11]:
pd.read_sql(
    '''
    WITH max_RM AS 
              (SELECT 
                     AVG(RM) AS avg_MAX_RM
                     
               FROM boston
               WHERE MEDV in (SELECT 
                                    MAX(MEDV) AS max_MEDV
                              FROM boston
                              )
                )
                 
    SELECT 
           AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 25 FOLLOWING) as 'avg_RM_25',
           AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 50 FOLLOWING) as 'avg_RM_50',
           AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 100 FOLLOWING) as 'avg_RM_100',
           AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 200 FOLLOWING) as 'avg_RM_200',
           AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 300 FOLLOWING) as 'avg_RM_300',
           avg_MAX_RM,
           avg_MAX_RM-AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 25 FOLLOWING) AS dif_avg_25,
           avg_MAX_RM-AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 50 FOLLOWING) AS dif_avg_50,
           avg_MAX_RM-AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 100 FOLLOWING) AS dif_avg_100,
           avg_MAX_RM-AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 200 FOLLOWING) AS dif_avg_200,
           avg_MAX_RM-AVG(RM) OVER (ORDER BY MEDV ROWS BETWEEN unbounded preceding AND 300 FOLLOWING) AS dif_avg_300
                
    FROM boston JOIN max_RM
    ORDER BY  MEDV    
    LIMIT 1
    
    ''',
    con,
)

,avg_RM_25,avg_RM_50,avg_RM_100,avg_RM_200,avg_RM_300,avg_MAX_RM,dif_avg_25,dif_avg_50,dif_avg_100,dif_avg_200,dif_avg_300
0,5.714462,5.757608,5.882832,5.913284,5.973425,7.484,1.769538,1.726392,1.601168,1.570716,1.510575


Во втором варианте отличается среднее количество комнат для самых дешевых 25 домов, потому что у 24,25,26 дома одинаковая стоимость: в первом варианте это учитывается, а во втором берется ровно 25 домов и считается среднее

↑ Чем больше количество комнта, тем выше стоимость дома

__Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.__

In [12]:
tasks+=1
pd.read_sql(
    '''     
    WITH rank_by_LSTATT AS
    (SELECT 
           LSTAT,
           MEDV,rank() OVER (ORDER BY LSTAT) rank_by_LSTAT
     FROM boston
     ) 
    
    SELECT 
           LSTAT,
           MEDV,
           rank_by_LSTAT          
    FROM rank_by_LSTATT
    WHERE MEDV in (SELECT 
                          MAX(boston.MEDV) AS max_MEDV
                   FROM boston
                  )
   
    ''',
    con,
)

,LSTAT,MEDV,rank_by_LSTAT
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


   ↑ По этим данным нельзя сделать какой-то однозначный вывод, поскольку мы видим, что процент населения с более низким статусом в самых дорогих домах так же присутствует, хотя и очевидно, что в этих домах он минимальный по сравнению с более дешевыми домами. Нужны дополнительные факторы

__Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)__

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT Distinct CHAS,
           AVG(MEDV) OVER (Partition BY CHAS) AS avg_MEDV_by_Chas
    FROM boston
    
    ''',
    con,
)

,CHAS,avg_MEDV_by_Chas
0,0.0,22.093843
1,1.0,28.440000


↑ Наличие водоёма рядом явно увеличивает стоимость жилья 

__Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.__


In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT Distinct CHAS,
           AVG(CRIM) OVER (Partition BY CHAS) AS avg_CRIM,
           AVG(ZN) OVER (Partition BY CHAS) AS avg_ZN,
           AVG(INDUS) OVER (Partition BY CHAS) AS avg_INDUS,
           AVG(NOX) OVER (Partition BY CHAS) AS avg_NOX
    FROM boston
       
    ''',
    con,
)

,CHAS,avg_CRIM,avg_ZN,avg_INDUS,avg_NOX
0,0.0,3.723215,11.634820,11.019193,0.551817
1,1.0,1.851670,7.714286,12.719143,0.593426


 В таблице ↑ мы видим, что при CHAS=1, среднее значение колонок INDUS и NOX - выше, чем при CHAS=0.
 А среднее значение колонок CRIM,ZN при CHAS=1 сильно ниже. 

Промышленность рядом с водоёмом более развита и концентрация оксидов азота более выраженная. 

Уровень преступности в два раза ниже в домах при водоёмах: возможно наличие воды, жилой земли и чистого воздуха  каким-то образом утихомиривает душу преступников. 

__Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцентиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь__

In [17]:
pd.read_sql(
    """
    with pin as (
        SELECT INDUS , PERCENT_RANK() OVER(ORDER BY INDUS ASC) as pin FROM boston
    ),
    pox as (
        SELECT NOX, PERCENT_RANK() OVER(ORDER BY NOX ASC) as pox FROM boston
    )
    SELECT 10 percintile, MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.1 and pox.pox <= 0.1
    UNION
    SELECT 20 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.2 and pox.pox <= 0.2
    UNION
    SELECT 30 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.3 and pox.pox <= 0.3
    UNION
    SELECT 40 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.4 and pox.pox <= 0.4
    UNION
    SELECT 50 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.5 and pox.pox <= 0.5
    UNION
    SELECT 60 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.6 and pox.pox <= 0.6
    UNION
    SELECT 70 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.7 and pox.pox <= 0.7
    UNION
    SELECT 80 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.8 and pox.pox <= 0.8
    UNION
    SELECT 90 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.9 and pox.pox <= 0.9
    UNION
    SELECT 100 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 1 and pox.pox <= 1
    

    
    
    limit 100
    """,
    con,
)

,percintile,indus,nox
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,12.83,0.575
6,70,18.10,0.605
7,80,18.10,0.668
8,90,19.58,0.713
9,100,27.74,0.871


С ростом промышленности растёт и NOX

In [16]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
